### Post processing text output of ID retrievals (gpt, llama)

In [ ]:
'''
>>> Script for post processing gpt and llama ID retrievals...
It takes text files and converts them to json files for evaluation.

'''

import re
import json
from pathlib import Path

results_txt = Path("retrievals/txt/llama4.scout_id_retrieval_nl.txt") # change name of the file [llama3.3.70b_id_retrieval_{lang}.txt], [gpt4.1.mini_pw_retrievals_nl.txt], [qwen3-235B_id_retrieval_nl.txt]
output_json = Path("retrievals/json/llama4.scout_id_retrieval_nl.json") # output

pattern_query = re.compile(r"^query id:\s*(\d+)")
pattern_relevant = re.compile(r"^relevant articles:\s*(.*)")

results_dict = {}

with open(results_txt, encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

i = 0
while i < len(lines) - 1:
    m_query = pattern_query.match(lines[i])
    m_relevant = pattern_relevant.match(lines[i+1])
    if m_query and m_relevant:
        qid = m_query.group(1)
        relevant_articles = [x.strip() for x in m_relevant.group(1).split(",") if x.strip()]
        results_dict[qid] = relevant_articles
        i += 2
    else:
        i += 1

with open(output_json, "w", encoding="utf-8") as out:
    json.dump(results_dict, out, indent=2, ensure_ascii=False)

print(f"results_nl.json written to: {output_json}")

### Post processing gpt binary-classification retrievals

In [ ]:
import json

input_path = "retrievals/llama4.scout_bin_full_class_retrievals_nl.jsonl"
output_path = "retrievals/llama4.scout_bin_class_retrieval_nl_fixed.jsonl"

buffer = ""
results = []

with open(input_path, "r", encoding="utf-8") as f:
    for line in f:
        buffer += line
        if line.strip().endswith("}"): 
            try:
                obj = json.loads(buffer)

                if "relevance" in obj and isinstance(obj["relevance"], dict):
                    normalized_relevance = {}
                    for k, v in obj["relevance"].items():
                        str_k = str(k)
                        str_v = str(v) if v in [0, 1, "0", "1"] else "?"
                        normalized_relevance[str_k] = str_v
                    obj["relevance"] = normalized_relevance

                results.append(obj)
                buffer = ""

            except json.JSONDecodeError:
                continue  

with open(output_path, "w", encoding="utf-8") as out_f:
    for obj in results:
        json_line = json.dumps(obj, ensure_ascii=False)
        out_f.write(json_line + "\n")

In [ ]:
'''
>>> Script for checking the LLM output. It checks if LLM changes or added or removed any article ids --> comparing to all hard negatives.
It also checks the values if they are only 0 or 1. No other characters or empty values. FOR GPT output...

'''

import json

gpt_output_path = "retrievals/llama4.scout_bin_class_retrieval_nl_fixed.jsonl"
hard_negatives_path = "../sampling_hard_negatives/hard_negatives/hard_negatives_nl.jsonl"

# load hard negatives
hard_negatives = {}
with open(hard_negatives_path, encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        hard_negatives[entry["query_id"]] = set(entry["candidate_docs"])

invalid_values_queries = []
missing_ids_queries = {}
extra_ids_queries = {}
valid_queries = []

with open(gpt_output_path, encoding="utf-8") as f:
    for idx, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            continue

        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            print(f"Line {idx}: INVALID JSON")
            continue

        query_id = obj.get("query_id")
        relevance = obj.get("relevance", {})

        # check relevance values
        invalid_values = [v for v in relevance.values() if v not in ("0", "1")]
        if invalid_values:
            invalid_values_queries.append(query_id)

        # check candidate IDs
        expected_ids = hard_negatives.get(query_id)
        if not expected_ids:
            continue

        actual_ids = set(relevance.keys())

        missing = expected_ids - actual_ids
        extra = actual_ids - expected_ids

        if missing:
            missing_ids_queries[query_id] = missing
        if extra:
            extra_ids_queries[query_id] = extra
        if not invalid_values and not missing and not extra:
            valid_queries.append(query_id)

print("\n=== SUMMARY ===")
print(f"Total queries processed: {len(valid_queries) + len(invalid_values_queries) + len(missing_ids_queries) + len(extra_ids_queries)}")
print(f"Fully correct queries: {len(valid_queries)}")

if invalid_values_queries:
    print(f"\nQueries with invalid relevance values: {invalid_values_queries}")
if missing_ids_queries:
    print("\nQueries with missing article IDs:")
    for qid, ids in missing_ids_queries.items():
        print(f"  Query {qid}: Missing IDs: {ids}")
if extra_ids_queries:
    print("\nQueries with extra article IDs:")
    for qid, ids in extra_ids_queries.items():
        print(f"  Query {qid}: Extra IDs: {ids}")

In [ ]:
'''
>>> Script for post ptocessing the binary classification (0/1) outputs and convert them to json retrievals. Only getting the 1 values --> relevant articles. 

'''

import json

input_path = "retrievals/llama4.scout_bin_class_retrieval_nl_fixed.jsonl"
output_path = "retrievals/json/llama4.scout_bin_class_retrieval_nl.json"

result = {}

with open(input_path, encoding="utf-8") as fin:
    for line in fin:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        query_id = obj["query_id"]
        relevance = obj["relevance"]

        relevant_articles = [aid for aid, val in relevance.items() if val == "1"]
        result[query_id] = relevant_articles

with open(output_path, "w", encoding="utf-8") as fout:
    json.dump(result, fout, indent=2, ensure_ascii=False)

### checking all retrievals, if all queries are present.

In [ ]:
import json
from pathlib import Path

# Paths — adjust if needed
#predictions_json = Path("retrievals/json/gemini_2.5_flash_id_retrieval_nl.json")
#predictions_json = Path("retrievals/json/gpt4.1.mini_id_retrievals_nl.json")
#predictions_json = Path("retrievals/json/gpt4o.mini_id_retrievals_nl.json")
#predictions_json = Path("retrievals/json/gpt4.1.mini_bin_class_retrievals_nl.json")
#predictions_json = Path("retrievals/json/gpt4o.mini_bin_class_retrievals_nl.json")
#predictions_json = Path("retrievals/json/gemini_2.5_flash_pro_id_retrieval_nl.json")
#predictions_json = Path("retrievals/json/gemini_2.5.flash_bin_class_retrieval_nl.json")
#predictions_json = Path("retrievals/json/gemini_2.5.pro_id_retrieval_nl.json")
#predictions_json = Path("retrievals/json/llama3.3.70b_id_retrieval_nl.json")
#predictions_json = Path("retrievals/json/llama3.3.70b_bin_class_retrieval_nl.json")
#predictions_json = Path("retrievals/json/qwen3.235b_bin_class_retrieval_nl.json")
predictions_json = Path("retrievals/json/llama4.scout_bin_class_retrieval_nl.json")

gold_json = Path("gold_data/gold_standard_nl.json")

# predictions
with open(predictions_json, encoding="utf-8") as f:
    predictions = json.load(f)

# gold
with open(gold_json, encoding="utf-8") as f:
    gold = json.load(f)

pred_ids = set(predictions.keys())
gold_ids = set(gold.keys())

missing_in_preds = gold_ids - pred_ids
extra_in_preds = pred_ids - gold_ids

print(f"Total gold queries: {len(gold_ids)}")
print(f"Total predicted queries: {len(pred_ids)}\n")

if missing_in_preds:
    print(f"Missing in predictions ({len(missing_in_preds)}):")
    for qid in sorted(missing_in_preds):
        print(f"  {qid}")
else:
    print("All gold query IDs are present in predictions.")

if extra_in_preds:
    print(f"\nExtra query IDs in predictions ({len(extra_in_preds)}):")
    for qid in sorted(extra_in_preds):
        print(f"  {qid}")
else:
    print("No extra query IDs in predictions.")

if not missing_in_preds and not extra_in_preds:
    print("\nPredictions file matches gold file perfectly.")
else:
    print("\nPlease fix mismatches before evaluating.")

## Checking the Output of Sorted Ranked Lists

#### Step 2: Converting .txt Files to jsonl Files for Evaluation

In [ ]:
import json
import re
from pathlib import Path

# -------- CONFIG --------
input_txt_path = Path("rankings/gpt4o-mini_sorted_ranking_nl.txt")
output_jsonl_path = Path("rankings/gpt4o-mini_sorted_ranking_nl.jsonl")

def extract_ids_any_format(ranked_field):
    """Extract article IDs from either a list, number string, or mixed format."""
    if isinstance(ranked_field, list):
        blob = " ".join(str(x) for x in ranked_field)
    else:
        blob = str(ranked_field)
    return re.findall(r"\d+", blob)

def get_field(obj, *possible_keys):
    for k in possible_keys:
        if k in obj:
            return obj[k]
    return None

jsonl_entries = []

with open(input_txt_path, "r", encoding="utf-8") as f:
    raw_text = f.read().strip()

parsed_qids = set()

# -------- TRY FULL JSON ARRAY PARSE --------
try:
    cleaned = re.sub(r"}\s*{", "},\n{", raw_text)
    cleaned = re.sub(r",\s*]", "]", cleaned)
    cleaned = re.sub(r",\s*}", "}", cleaned)
    if not cleaned.strip().startswith("["):
        cleaned = "[" + cleaned + "]"
    data = json.loads(cleaned)
    for obj in data:
        qid_raw = get_field(obj, "query_id", "query id")
        ranks_raw = get_field(obj, "ranked_articles", "ranked articles")
        if qid_raw is None:
            continue
        query_id = str(qid_raw).strip().strip('"').strip("'")
        ranks_list = extract_ids_any_format(ranks_raw)
        jsonl_entries.append({"query_id": query_id, "ranks": ranks_list})
        parsed_qids.add(query_id)
    print("Parsed using full JSON array method.")
except json.JSONDecodeError:
    print("Full JSON array parse failed, continuing...")

# -------- TRY JSON OBJECT REGEX EXTRACTION --------
json_objects = re.findall(r"\{.*?\}", raw_text, flags=re.DOTALL)
for obj_str in json_objects:
    try:
        obj = json.loads(obj_str)
    except json.JSONDecodeError:
        continue
    qid_raw = get_field(obj, "query_id", "query id")
    ranks_raw = get_field(obj, "ranked_articles", "ranked articles")
    if qid_raw is None:
        continue
    query_id = str(qid_raw).strip().strip('"').strip("'")
    if query_id in parsed_qids:
        continue
    ranks_list = extract_ids_any_format(ranks_raw)
    jsonl_entries.append({"query_id": query_id, "ranks": ranks_list})
    parsed_qids.add(query_id)
print("Parsed using JSON object extraction method (where possible).")

# -------- FALLBACK: RAW TEXT SCAN FOR ANY REMAINING QIDS --------
current_qid = None
current_ids = []
for line in raw_text.splitlines():
    line = line.strip()
    if not line:
        continue
    qid_match = re.search(r'"?query[_ ]id"?\s*:\s*"?(\d+)"?', line, re.IGNORECASE)
    if qid_match:
        if current_qid and current_qid not in parsed_qids:
            jsonl_entries.append({"query_id": current_qid, "ranks": current_ids})
            parsed_qids.add(current_qid)
        current_qid = qid_match.group(1)
        current_ids = []
        continue
    ids_in_line = re.findall(r"\d+", line)
    if ids_in_line:
        current_ids.extend(ids_in_line)
# flush last
if current_qid and current_qid not in parsed_qids:
    jsonl_entries.append({"query_id": current_qid, "ranks": current_ids})
    parsed_qids.add(current_qid)

# -------- SAVE JSONL --------
with open(output_jsonl_path, "w", encoding="utf-8") as f_out:
    for entry in jsonl_entries:
        f_out.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"Converted {len(jsonl_entries)} queries to {output_jsonl_path}")

### Step 3: Evaluation

In [ ]:
import json
from pathlib import Path
import math
from tqdm import tqdm
from statistics import mean

'''
>>> scored_ranking Files
'''
#predictions_json = Path("rankings/scored/json/gpt4.1.mini.ranks.nl.jsonl") 
#predictions_json = Path("rankings/scored/json/gpt4o.mini.ranks.nl.jsonl") 

#predictions_json = Path("rankings/scored/json/gemini2.5.flash.ranks.nl.jsonl") 

#predictions_json = Path("rankings/scored/json/qwen3.235b.ranks.nl.jsonl") 
#predictions_json = Path("rankings/scored/json/llama3.3.70b.ranks.nl.jsonl") 
#predictions_json = Path("rankings/scored/json/llama4.scout.ranks.nl.jsonl") 


'''
>>> Sorted_ranking Files
'''
#predictions_json = Path("rankings/sorted/json/gemini2.5.flash_sorted_ranks_nl.jsonl") 
#predictions_json = Path("rankings/sorted/json/gpt4.1.mini_sorted_ranks_nl.jsonl") 
#predictions_json = Path("rankings/sorted/json/gpt4o-mini_sorted_ranks_nl.jsonl") 
#predictions_json = Path("rankings/sorted/json/qwen3.235b_sorted_ranks_nl.jsonl") 
#predictions_json = Path("rankings/sorted/json/llama4_scout_sorted_ranks_nl.jsonl") 
#predictions_json = Path("rankings/sorted/json/llama3.3_70b_sorted_ranks_nl.jsonl") 


'''
>>> Baselines
'''
#predictions_json = Path("rankings/sorted/json/me5_top100_ranks_nl.jsonl")
predictions_json = Path("rankings/sorted/json/jina_ranks_nl.jsonl")


gold_json = Path("gold_data/gold_standard_nl.json")
output_dir = Path("evaluation")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "eval_sorted_ranking_jina.txt" 

ks = [1, 5, 10, 20, 50, 100]

# load predictions from JSONL
predictions = {}
with open(predictions_json, encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        predictions[item["query_id"]] = item["ranks"]

# load gold
with open(gold_json, encoding="utf-8") as f:
    gold = json.load(f)

print(f"Loaded predictions for {len(predictions)} queries")
print(f"Loaded gold standard for {len(gold)} queries")

metrics = {f"R@{k}": [] for k in ks}
metrics.update({f"MRR@{k}": [] for k in ks})
metrics.update({f"MAP@{k}": [] for k in ks})
metrics.update({f"nDCG@{k}": [] for k in ks})

def compute_hits(relevant, predicted, k):
    hits = [1 if doc in relevant else 0 for doc in predicted[:k]]
    return hits

for qid in tqdm(predictions.keys(), desc="Evaluating"):
    pred = predictions[qid]
    gold_set = set(gold[qid])

    for k in ks:
        hits = compute_hits(gold_set, pred, k)

        recall = sum(hits) / len(gold_set) if gold_set else 0.0
        metrics[f"R@{k}"].append(recall)

        rr = 0.0
        for rank, h in enumerate(hits):
            if h:
                rr = 1.0 / (rank + 1)
                break
        metrics[f"MRR@{k}"].append(rr)

        ap = 0.0
        hit_count = 0
        for rank, h in enumerate(hits):
            if h:
                hit_count += 1
                ap += hit_count / (rank + 1)
        ap /= len(gold_set) if gold_set else 1
        metrics[f"MAP@{k}"].append(ap)

        dcg = 0.0
        for i, h in enumerate(hits):
            if h:
                dcg += 1.0 / (math.log2(i + 2))  # +2 because log2(rank+1), and rank = 0-based

        ideal_hits = [1] * min(len(gold_set), k)
        idcg = sum(1.0 / math.log2(i + 2) for i in range(len(ideal_hits)))
        ndcg = dcg / idcg if idcg > 0 else 0.0
        metrics[f"nDCG@{k}"].append(ndcg)

print()
with open(output_file, "w", encoding="utf-8") as out:
    for m, vals in metrics.items():
        line = f"{m}: {mean(vals):.4f}"
        print(line)
        out.write(line + "\n")

print(f"\nEvaluation results saved to: {output_file}")

In [ ]:
import json
import math
from pathlib import Path
from tqdm import tqdm
from statistics import mean
import pandas as pd

prediction_files = [
    "rankings/scored/json/gpt4.1.mini.ranks.nl.jsonl",
    "rankings/scored/json/gpt4o.mini.ranks.nl.jsonl",
    "rankings/scored/json/gemini2.5.flash.ranks.nl.jsonl",
    "rankings/scored/json/qwen3.235b.ranks.nl.jsonl",
    "rankings/scored/json/llama3.3.70b.ranks.nl.jsonl",
    "rankings/scored/json/llama4.scout.ranks.nl.jsonl",
    "rankings/sorted/json/gemini2.5.flash_sorted_ranks_nl.jsonl",
    "rankings/sorted/json/gpt4.1.mini_sorted_ranks_nl.jsonl",
    "rankings/sorted/json/gpt4o-mini_sorted_ranks_nl.jsonl",
    "rankings/sorted/json/qwen3.235b_sorted_ranks_nl.jsonl",
    "rankings/sorted/json/llama4_scout_sorted_ranks_nl.jsonl",
    "rankings/sorted/json/llama3.3_70b_sorted_ranks_nl.jsonl",
    "rankings/sorted/json/me5_top100_ranks_nl.jsonl",
    "rankings/sorted/json/jina_ranks_nl.jsonl"
]

gold_json = Path("gold_data/gold_standard_nl.json")
output_dir = Path("evaluation")
output_dir.mkdir(parents=True, exist_ok=True)

output_txt = output_dir / "eval_all_models.txt"
output_csv = output_dir / "eval_all_models.csv"

ks = [1, 5, 10, 20, 50, 100]

with open(gold_json, encoding="utf-8") as f:
    gold = json.load(f)

def compute_hits(relevant, predicted, k):
    return [1 if doc in relevant else 0 for doc in predicted[:k]]

results_table = []

for file_path in prediction_files:
    file_path = Path(file_path)
    model_name = file_path.stem 

    predictions = {}
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            predictions[item["query_id"]] = item["ranks"]

    print(f"Evaluating {model_name}: {len(predictions)} queries loaded.")

    metrics = {f"R@{k}": [] for k in ks}
    metrics.update({f"MRR@{k}": [] for k in ks})
    metrics.update({f"MAP@{k}": [] for k in ks})
    metrics.update({f"nDCG@{k}": [] for k in ks})

    for qid in tqdm(predictions.keys(), desc=f"Evaluating {model_name}"):
        pred = predictions[qid]
        gold_set = set(gold[qid])

        for k in ks:
            hits = compute_hits(gold_set, pred, k)

            recall = sum(hits) / len(gold_set) if gold_set else 0.0
            metrics[f"R@{k}"].append(recall)

            rr = 0.0
            for rank, h in enumerate(hits):
                if h:
                    rr = 1.0 / (rank + 1)
                    break
            metrics[f"MRR@{k}"].append(rr)

            ap = 0.0
            hit_count = 0
            for rank, h in enumerate(hits):
                if h:
                    hit_count += 1
                    ap += hit_count / (rank + 1)
            ap /= len(gold_set) if gold_set else 1
            metrics[f"MAP@{k}"].append(ap)

            dcg = 0.0
            for i, h in enumerate(hits):
                if h:
                    dcg += 1.0 / (math.log2(i + 2))
            ideal_hits = [1] * min(len(gold_set), k)
            idcg = sum(1.0 / math.log2(i + 2) for i in range(len(ideal_hits)))
            ndcg = dcg / idcg if idcg > 0 else 0.0
            metrics[f"nDCG@{k}"].append(ndcg)

    results_table.append({
        "Model": model_name,
        "R@1": mean(metrics["R@1"]),
        "R@5": mean(metrics["R@5"]),
        "R@10": mean(metrics["R@10"]),
        "MRR@10": mean(metrics["MRR@10"]),
        "MAP@10": mean(metrics["MAP@10"]),
        "nDCG@1": mean(metrics["nDCG@1"]),
        "nDCG@10": mean(metrics["nDCG@10"]),
        "nDCG@100": mean(metrics["nDCG@100"]),
    })

# Save TXT
with open(output_txt, "w", encoding="utf-8") as f:
    header = "Model\tR@1\tR@5\tR@10\tMRR@10\tMAP@10\tnDCG@1\tnDCG@10\tnDCG@100\n"
    f.write(header)
    for row in results_table:
        f.write(
            f"{row['Model']}\t"
            f"{row['R@1']:.4f}\t{row['R@5']:.4f}\t{row['R@10']:.4f}\t"
            f"{row['MRR@10']:.4f}\t{row['MAP@10']:.4f}\t"
            f"{row['nDCG@1']:.4f}\t{row['nDCG@10']:.4f}\t{row['nDCG@100']:.4f}\n"
        )

df = pd.DataFrame(results_table)
df.to_csv(output_csv, index=False)

## chacking and post-processing the scored_ranking results

In [ ]:
import os
import json
from pathlib import Path
from tqdm import tqdm

output_path = Path(f"rankings/scored/llama4.scout_score_ranking_nl.jsonl")
hard_negatives_path = Path(f"../sampling_hard_negatives/hard_negatives/hard_negatives_nl.jsonl")
fixed_output_path = Path(f"rankings/scored/llama4.scout_score_ranking_nl_sorted.jsonl")


with open(hard_negatives_path, "r", encoding="utf-8") as f:
    hard_negatives = [json.loads(line) for line in f]

expected_query_ids = set()
expected_doc_ids_per_query = {}

for entry in hard_negatives:
    qid = str(entry["query_id"])
    expected_query_ids.add(qid)
    expected_doc_ids_per_query[qid] = set(entry["candidate_docs"])


fixed_entries = []
current_entry_lines = []

with open(output_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line.startswith('{') and current_entry_lines:
            try:
                entry = json.loads(' '.join(current_entry_lines))
                fixed_entries.append(entry)
            except json.JSONDecodeError:
                print("Skipping malformed entry.")
            current_entry_lines = [line]
        else:
            current_entry_lines.append(line)

    if current_entry_lines:
        try:
            entry = json.loads(' '.join(current_entry_lines))
            fixed_entries.append(entry)
        except json.JSONDecodeError:
            print("Skipping final malformed entry.")

valid_entries = []

for entry in tqdm(fixed_entries, desc="Validating and sorting"):
    qid = entry.get("query_id")
    scores = entry.get("relevance_scores", {})
    if not qid or qid not in expected_query_ids:
        print(f"Skipping unknown or missing query ID: {qid}")
        continue

    if set(scores.keys()) != expected_doc_ids_per_query[qid]:
        print(f"Skipping query {qid}: mismatched article IDs.")
        continue

    try:
        int_scores = {doc_id: int(score) for doc_id, score in scores.items()}
    except ValueError:
        print(f"Skipping query {qid}: contains non-integer-convertible score.")
        continue

    sorted_scores = dict(sorted(int_scores.items(), key=lambda x: -x[1]))
    valid_entries.append({
        "query_id": qid,
        "relevance_scores": sorted_scores
    })


fixed_output_path.parent.mkdir(parents=True, exist_ok=True)
with open(fixed_output_path, "w", encoding="utf-8") as f_out:
    for entry in valid_entries:
        f_out.write(json.dumps(entry) + "\n")

print(f"\nFixed and validated output written to: {fixed_output_path}")

ranks_only_output_path = Path("rankings/scored/llama4.scout.ranks.nl.jsonl")

with open(ranks_only_output_path, "w", encoding="utf-8") as f_out:
    for entry in valid_entries:
        qid = entry["query_id"]
        ranked_ids = list(entry["relevance_scores"].keys())  
        f_out.write(json.dumps({
            "query_id": qid,
            "ranks": ranked_ids
        }) + "\n")

print(f"Ranks-only output written to: {ranks_only_output_path}")

fixed_query_ids = {entry["query_id"] for entry in valid_entries}
missing_query_ids = sorted(expected_query_ids - fixed_query_ids)

if missing_query_ids:
    print(f"\n⚠ Missing query IDs ({len(missing_query_ids)}):")
    print(missing_query_ids)
else:
    print("\nAll query IDs accounted for.")

In [ ]:
import os
import json
import pandas as pd

evaluation_folder = "retrievals/evaluation"

files = [f for f in os.listdir(evaluation_folder) if f.endswith(".txt")]

macro_data = []
micro_data = []

def parse_score(value):
    num = float(value)
    if num > 1.0 and num <= 100:
        num = num / 1000  
    return round(num, 2) 

for filename in files:
    filepath = os.path.join(evaluation_folder, filename)
    
    with open(filepath, "r") as f:
        lines = [line.strip() for line in f if line.strip()]
    
    macro_index = lines.index("Macro-averaged metrics:")
    micro_index = lines.index("Micro-averaged metrics:")
    
    macro_precision = parse_score(lines[macro_index + 1].split(":")[1].strip())
    macro_recall = parse_score(lines[macro_index + 2].split(":")[1].strip())
    macro_f1 = parse_score(lines[macro_index + 3].split(":")[1].strip())
    
    micro_precision = parse_score(lines[micro_index + 1].split(":")[1].strip())
    micro_recall = parse_score(lines[micro_index + 2].split(":")[1].strip())
    micro_f1 = parse_score(lines[micro_index + 3].split(":")[1].strip())

    name = filename.replace(".txt", "")
    if "id_retr_" in name:
        method = "ID-Retrieval"
        model = name.split("id_retr_")[-1]
    elif "bin_retr_" in name:
        method = "Relevance-Classification"
        model = name.split("bin_retr_")[-1]
    else:
        method = "Unknown"
        model = name

    macro_data.append({
        "Model": model,
        "Method": method,
        "Precision": macro_precision,
        "Recall": macro_recall,
        "F1-Score": macro_f1
    })

    micro_data.append({
        "Model": model,
        "Method": method,
        "Precision": micro_precision,
        "Recall": micro_recall,
        "F1-Score": micro_f1
    })

df_macro = pd.DataFrame(macro_data)
df_micro = pd.DataFrame(micro_data)

df_macro.to_csv(os.path.join(evaluation_folder, "macro_scores.csv"), index=False, float_format="%.2f")
df_micro.to_csv(os.path.join(evaluation_folder, "micro_scores.csv"), index=False, float_format="%.2f")